In [5]:
import pandas as pd
import os
#from PyPDF2 import PdfReader
import glob
from bs4 import BeautifulSoup

In [6]:
Scrapes_NoFull_df = pd.read_csv('Scrapes_ALL.csv')
print('We have collected', Scrapes_NoFull_df.shape[0], 'FER classification papers')

# Function to extract the base domain of a URL (until .com, .org, etc.)
def extract_base_domain(url):
    if pd.notna(url):
        # Split the URL and return the main domain (e.g., 'arxiv.org', 'ieee.org')
        return '.'.join(url.split('/')[2].split('.')[-2:])
    return None

# Apply the function to extract base domains
Scrapes_NoFull_df['Base_Domain'] = Scrapes_NoFull_df['URL'].apply(extract_base_domain)

Scrapes_NoFull_df['Base_Domain'].value_counts()

We have collected 261 FER classification papers


Base_Domain
ieee.org              84
arxiv.org             41
sciencedirect.com     40
springer.com          22
nih.gov               22
acm.org               13
mdpi.com               8
thecvf.com             6
researchgate.net       4
paperswithcode.com     2
wiley.com              2
tandfonline.com        2
psu.edu                1
cmu.edu                1
oup.com                1
computer.org           1
kdef.se                1
auth.gr                1
mlr.press              1
iop.org                1
utpjournals.press      1
github.com             1
cedus.it               1
nature.com             1
sagepub.com            1
plos.org               1
toronto.edu            1
Name: count, dtype: int64

# ieee read in

In [8]:
import pandas as pd

# Path to the csv file
file_path_IEEE = 'results/ieee_full_texts.csv'

# Read the csv file
FullTextIEEE_df = pd.read_csv(file_path_IEEE)

# Replace specified strings with actual NaN values
FullTextIEEE_df['texts'] = FullTextIEEE_df['texts'].replace(["Nothing found", "Error"], pd.NA)

# Count entries where 'texts' is not NaN
found_text_count = FullTextIEEE_df['texts'].notna().sum()

print(f"Number of entries with extracted text: {found_text_count}")


Number of entries with extracted text: 65


# arxiv full texts to csv 

In [9]:
#read in the csv file with the saved arxiv fulltext
file_path_Arxiv = 'results/arxiv_full_texts.csv'

FullTextArx_df = pd.read_csv(file_path_Arxiv)

columns_to_remove = ['Unnamed: 0.1', 'Unnamed: 0', 'DOI', 'Mentions_Accuracy', 'Mentions_F1', 'Mentions_Precision', 'Mentions_Recall', 'Mentions_Auc', 'Mentions_Roc', 'Mentions_Sensitivity', 'Mentions_Specificity', 'Mentions_Confusion_matrix', 'Mentions_Loss_function', 'Mentions_Cross-entropy', 'Mentions_Mean_squared_error', 'Mentions_Overfitting', 'Mentions_Underfitting', 'Mentions_Cross-validation', 'Mentions_Training_time', 'Mentions_Inference_time', 'Mentions_Statistical_significance', 'Mentions_P-value', 'Mentions_T-test', 'Mentions_Anova', 'Mentions_Correlation', 'Mentions_Regression', 'Mentions_Baseline_comparison', 'Mentions_Mae', 'Mentions_Rmse', 'Mentions_Bias'] 
# Drop the specified columns
FullTextArx_df = FullTextArx_df.drop(columns=columns_to_remove)

print(f"Number of entries with extracted text: {FullTextArx_df['Full_Text'].count()}")
#FullTextArx_df.head()

Number of entries with extracted text: 37


# SciencDirect read in

In [11]:
#read in the csv file with the saved arxiv fulltext
file_path_SD = 'results/science_direct_full_texts.csv'

FullTextSD_df = pd.read_csv(file_path_SD)

columns_to_remove = ['Unnamed: 0', 'DOI', 'Mentions_Accuracy', 'Mentions_F1', 'Mentions_Precision', 'Mentions_Recall', 'Mentions_Auc', 'Mentions_Roc', 'Mentions_Sensitivity', 'Mentions_Specificity', 'Mentions_Confusion_matrix', 'Mentions_Loss_function', 'Mentions_Cross-entropy', 'Mentions_Mean_squared_error', 'Mentions_Overfitting', 'Mentions_Underfitting', 'Mentions_Cross-validation', 'Mentions_Training_time', 'Mentions_Inference_time', 'Mentions_Statistical_significance', 'Mentions_P-value', 'Mentions_T-test', 'Mentions_Anova', 'Mentions_Correlation', 'Mentions_Regression', 'Mentions_Baseline_comparison', 'Mentions_Mae', 'Mentions_Rmse', 'Mentions_Bias'] 
# Drop the specified columns
FullTextSD_df = FullTextSD_df.drop(columns=columns_to_remove)

print(f"Number of entries with extracted text: {FullTextSD_df['Full_Text'].count()}")
#FullTextSD_df.head()

Number of entries with extracted text: 8


# Merge

In [15]:
# Assuming FullTextSD_df and FullTextArx_df are your two dataframes

# Fill NaNs in FullTextSD_df's 'Full_Text' column with corresponding values from FullTextArx_df's 'Full_Text' column
FullTextCombined_df = FullTextSD_df.copy()
FullTextCombined_df['Full_Text'] = FullTextCombined_df['Full_Text'].combine_first(FullTextArx_df['Full_Text'])
print(FullTextCombined_df['Full_Text'].count())

# Adjusting the merge to use 'URL' as the matching column instead of 'links'
FullTextCombined_df = FullTextCombined_df.merge(
    FullTextIEEE_df[['links', 'texts']].rename(columns={'links': 'URL'}),
    on='URL',
    how='left'
)
FullTextCombined_df.head()
# Use the 'texts_IEEE' column to fill NaN values in 'Full_Text' column of FullTextCombined_df
FullTextCombined_df['Full_Text'] = FullTextCombined_df['Full_Text'].combine_first(FullTextCombined_df['texts'])

# Drop the helper 'texts_IEEE' column
FullTextCombined_df = FullTextCombined_df.drop(columns=['texts'])

print(f"Number of entries with extracted text: {FullTextCombined_df['Full_Text'].count()}")
FullTextCombined_df = FullTextCombined_df.drop(['ID'], axis=1)
FullTextCombined_df.head()

44
Number of entries with extracted text: 96


,Title,Authors,Year,Cited By,Detected_Dataset,Detected_Topic,Abstract,Journal,URL,Full_Text
0,10 Automated Face Analysis for Affective Compu...,"['JF Cohn', 'F De la Torre']",2015,170,Affective Faces Database,classifier,Differences in manual coding between databases...,The Oxford handbook of affective …,https://academic.oup.com/edited-volume/28057/c...,NaN
1,3D facial expression recognition based on auto...,"['H Tang', 'TS Huang']",2008,205,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",facial expression recognition from 3D facial s...,… on computer vision and pattern recognition …,https://ieeexplore.ieee.org/document/4563052,3D Facial Expression Recognition Based on Auto...
2,3D facial expression recognition based on prim...,"['J Wang', 'L Yin', 'X Wei', 'Y Sun']",2006,440,Binghamton University 3D Facial Expression,facial expression recognition,expressions using 3D facial expression range d...,… Vision and Pattern Recognition …,https://ieeexplore.ieee.org/document/1640921,3D Facial Expression Recognition Based on Prim...
3,3D facial expression recognition based on prop...,"['H Tang', 'TS Huang']",2008,153,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",Binghamton University have recently constructe...,… on Automatic Face & Gesture Recognition,https://ieeexplore.ieee.org/document/4813304,3DFacial Expr ession Recognition Based onPrope...
4,3D facial expression recognition using SIFT de...,"['S Berretti', 'B Ben Amor', 'M Daoudi', 'A De...",2011,184,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",at the Binghamton University (BU-3DFE database...,The Visual Computer,https://www.researchgate.net/publication/22006...,NaN


In [16]:
FullTextCombined_df.columns

Index(['Title', 'Authors', 'Year', 'Cited By', 'Detected_Dataset',
       'Detected_Topic', 'Abstract', 'Journal', 'URL', 'Full_Text'],
      dtype='object')

In [17]:
FullTextCombined_df.to_csv('FullText_ALL.csv')